## PRIMERA VERSION PARA DESPLIEGUE 

## segunda version 

In [3]:
import pandas as pd
import re
import dash
from dash import dcc, html, Input, Output, State, dash_table
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
import numpy as np # Importar numpy para manejar np.nan y np.NA

# =============================================
# 1. CARGA DE DATOS (MODIFICADO PARA MULTI-HOJA EN ALQUILER)
# =============================================
rutas_archivos = [
    r"C:\Users\avillarreal\FUTURA RETAIL, C.A\Freddy Cohen - FINANZAS 2025\DASHBOARD\CONTROL ALQUILER.xlsx",
    r"C:\Users\avillarreal\FUTURA RETAIL, C.A\Freddy Cohen - FINANZAS 2025\DASHBOARD\CONDOMINIOS.xlsx",
    r"C:\Users\avillarreal\FUTURA RETAIL, C.A\Freddy Cohen - FINANZAS 2025\DASHBOARD\BBDD_VENTAS.xlsx",
    r"C:\Users\avillarreal\FUTURA RETAIL, C.A\Freddy Cohen - FINANZAS 2025\DASHBOARD\ALCALDIAS.xlsx",
    r"C:\Users\avillarreal\FUTURA RETAIL, C.A\Freddy Cohen - FINANZAS 2025\DASHBOARD\1-Gastos de Personal x  Razón Social 2025.xlsx",
]

nombres_archivos = [
    "CONTROL ALQUILER.xlsx",
    "CONDOMINIOS.xlsx",
    "BBDD_VENTAS.xlsx",
    "ALCALDIAS.xlsx",
    "1-Gastos de Personal x  Razón Social 2025.xlsx"
]

dataframes = {}

for i, ruta in enumerate(rutas_archivos):
    nombre_archivo_actual = nombres_archivos[i]
    try:
        if nombre_archivo_actual == "CONTROL ALQUILER.xlsx":
            print(f"Cargando todas las hojas de: {nombre_archivo_actual}")
            excel_file = pd.ExcelFile(ruta, engine='openpyxl')
            all_sheets_dict = excel_file.parse(sheet_name=None) 
            all_sheets_list = []
            for sheet_name, sheet_df in all_sheets_dict.items():
                print(f"  - Procesando hoja: {sheet_name}")
                if sheet_df is not None and not sheet_df.empty:
                    print(f"    Columnas encontradas en hoja '{sheet_name}': {sheet_df.columns.tolist()}")
                    all_sheets_list.append(sheet_df)
                else:
                    print(f"    * Hoja '{sheet_name}' está vacía o no se pudo cargar.")
            if all_sheets_list:
                combined_df = pd.concat(all_sheets_list, ignore_index=True)
                dataframes[nombre_archivo_actual] = combined_df
                print(f"Archivo '{nombre_archivo_actual}' cargado y hojas combinadas exitosamente. Filas totales: {len(combined_df)}")
                print(f"  Columnas en DataFrame combinado de Alquiler: {combined_df.columns.tolist()}")
            else:
                print(f"Advertencia: No se encontraron hojas válidas en '{nombre_archivo_actual}'.")
                dataframes[nombre_archivo_actual] = pd.DataFrame()
        else:
            df = pd.read_excel(ruta, engine='openpyxl')
            dataframes[nombre_archivo_actual] = df
            print(f"Archivo '{nombre_archivo_actual}' cargado exitosamente (primera hoja).")
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en la ruta: {ruta}")
        dataframes[nombre_archivo_actual] = pd.DataFrame() 
    except Exception as e:
        print(f"Ocurrió un error al leer el archivo {ruta}: {e}")
        dataframes[nombre_archivo_actual] = pd.DataFrame() 

print("\n--- DataFrames cargados en el diccionario 'dataframes' ---")
print(dataframes.keys())

# =============================================
# 2. FUNCIONES DE LIMPIEZA Y CONVERSIÓN
# =============================================
def limpiar_columna_monto_alternativa(serie_original):
    if serie_original is None or serie_original.empty:
        return pd.Series(dtype='float64')
    serie = serie_original.copy()
    serie_numerica_directa = pd.to_numeric(serie, errors='coerce')
    necesitan_limpieza_mask = serie_numerica_directa.isna() & serie_original.notna()
    valores_a_limpiar_str = serie[necesitan_limpieza_mask].astype(str)
    if not valores_a_limpiar_str.empty:
        serie_limpia_str = valores_a_limpiar_str.str.strip()
        serie_limpia_str = serie_limpia_str.str.replace('Bs.', '', regex=False)
        serie_limpia_str = serie_limpia_str.str.replace('$', '', regex=False)
        serie_limpia_str = serie_limpia_str.str.replace('.', '', regex=False) 
        serie_limpia_str = serie_limpia_str.str.replace(',', '.', regex=False) 
        serie_limpia_str = serie_limpia_str.str.replace(r'[^\d.-]', '', regex=True)
        valores_limpiados_numericos = pd.to_numeric(serie_limpia_str, errors='coerce')
        fallan_conversion_mask = valores_limpiados_numericos.isna() & (serie_limpia_str != '') & (serie_limpia_str.notna())
        if fallan_conversion_mask.any():
             print(f"\n--- DEBUG LIMPIEZA MONTO (Alternativa): Fallaron conversión en '{serie_original.name}': ---")
             print("Valores originales:")
             print(serie_original[necesitan_limpieza_mask][fallan_conversion_mask])
             print("Valores después de limpieza:")
             print(serie_limpia_str[fallan_conversion_mask])
             print("------------------------------------------------------------------\n")
        serie_numerica_directa.loc[necesitan_limpieza_mask] = valores_limpiados_numericos
    return serie_numerica_directa.fillna(0)

def convertir_nombre_mes_a_formato_yyyymm(serie_mes_nombre, año_defecto=2025):
    if serie_mes_nombre is None or serie_mes_nombre.empty:
        return pd.Series(dtype='object')
    mapa_meses = {
        'ENERO': 1, 'FEBRERO': 2, 'MARZO': 3, 'ABRIL': 4,
        'MAYO': 5, 'JUNIO': 6, 'JULIO': 7, 'AGOSTO': 8,
        'SEPTIEMBRE': 9, 'OCTUBRE': 10, 'NOVIEMBRE': 11, 'DICIEMBRE': 12
    }
    def convertir_valor(valor):
        if pd.isna(valor) or valor is None: return 'Desconocido' 
        valor_str = str(valor).upper().strip()
        if re.match(r'^\d{4}-\d{2}$', valor_str): return valor_str
        match_mes_año = re.match(r'([A-Z]+)[\s_-]*(\d{4})', valor_str) 
        if match_mes_año:
            nombre_mes, año_extraido = match_mes_año.groups()
            num_mes = mapa_meses.get(nombre_mes)
            if num_mes: return f"{año_extraido}-{num_mes:02d}"
        num_mes = mapa_meses.get(valor_str) 
        if num_mes: return f"{año_defecto}-{num_mes:02d}"
        if isinstance(valor, (int, float)) and valor > 10000: 
            try:
                dt_obj = pd.to_datetime('1899-12-30') + pd.to_timedelta(valor, 'D')
                return dt_obj.strftime('%Y-%m')
            except: pass 
        return 'Desconocido' 
    return serie_mes_nombre.apply(convertir_valor)

# =============================================
# 3. PREPARACIÓN DE DATOS PARA EL DASHBOARD
# =============================================
# --- Procesamiento de Ventas ---
if 'BBDD_VENTAS.xlsx' in dataframes and dataframes['BBDD_VENTAS.xlsx'] is not None and not dataframes['BBDD_VENTAS.xlsx'].empty:
    df_ventas = dataframes['BBDD_VENTAS.xlsx'].copy()
    print("\n--- Procesando DataFrame 'BBDD_VENTAS.xlsx' ---")
    if 'FECHA' in df_ventas.columns:
        df_ventas['FECHA'] = pd.to_datetime(df_ventas['FECHA'], errors='coerce')
        df_ventas['MES'] = df_ventas['FECHA'].dt.strftime('%Y-%m')
    elif 'MES' in df_ventas.columns:
        df_ventas['MES'] = convertir_nombre_mes_a_formato_yyyymm(df_ventas['MES'])
    else: df_ventas['MES'] = 'Desconocido'
    
    df_ventas['Ventas $'] = limpiar_columna_monto_alternativa(df_ventas['Ventas $']) if 'Ventas $' in df_ventas.columns else 0
    df_ventas['Coste $'] = limpiar_columna_monto_alternativa(df_ventas['Coste $']) if 'Coste $' in df_ventas.columns else 0

    marca_col_ventas = next((col for col in ['Marca', 'MARCA'] if col in df_ventas.columns), None)
    if marca_col_ventas:
        if marca_col_ventas != 'Marca': df_ventas.rename(columns={marca_col_ventas: 'Marca'}, inplace=True)
        df_ventas['Marca'] = df_ventas['Marca'].fillna('DESCONOCIDA').astype(str).str.upper().str.strip()
    else: df_ventas['Marca'] = 'DESCONOCIDA'

    ubicacion_col_ventas = next((col for col in ['UBICACION', 'Ubicacion'] if col in df_ventas.columns), None)
    if ubicacion_col_ventas:
        if ubicacion_col_ventas != 'UBICACION': df_ventas.rename(columns={ubicacion_col_ventas: 'UBICACION'}, inplace=True)
        df_ventas['UBICACION'] = df_ventas['UBICACION'].fillna('DESCONOCIDA').astype(str).str.upper().str.strip()
    else: df_ventas['UBICACION'] = 'DESCONOCIDA'
else:
    df_ventas = pd.DataFrame(columns=['Marca', 'UBICACION', 'Ventas $', 'Coste $', 'MES'])

# --- Procesamiento de Nómina ---
nomina_key = '1-Gastos de Personal x  Razón Social 2025.xlsx'
columnas_nomina_sumar = ['SUELDOS DIRECTIVOS$', 'SUELDOS $', 'CESTATICKET $', 'AYUDA $', 'COMISIONES $', 'OTROS $', 'UTILIDADES $', 'VACACIONES $']
if nomina_key in dataframes and dataframes[nomina_key] is not None and not dataframes[nomina_key].empty:
    df_nomina = dataframes[nomina_key].copy()
    
    marca_col_nomina = next((col for col in ['MARCA', 'Marca'] if col in df_nomina.columns), None)
    if marca_col_nomina:
        if marca_col_nomina != 'MARCA': df_nomina.rename(columns={marca_col_nomina: 'MARCA'}, inplace=True)
        df_nomina['MARCA'] = df_nomina['MARCA'].fillna('DESCONOCIDA').astype(str).str.upper().str.strip()
    else: df_nomina['MARCA'] = 'DESCONOCIDA'

    ubicacion_col_nomina = next((col for col in ['UBICACION', 'Ubicacion'] if col in df_nomina.columns), None)
    if ubicacion_col_nomina:
        if ubicacion_col_nomina != 'UBICACION': df_nomina.rename(columns={ubicacion_col_nomina: 'UBICACION'}, inplace=True)
        df_nomina['UBICACION'] = df_nomina['UBICACION'].fillna('DESCONOCIDA').astype(str).str.upper().str.strip()
    else: df_nomina['UBICACION'] = 'DESCONOCIDA'

    if 'MES' in df_nomina.columns:
        df_nomina['MES'] = convertir_nombre_mes_a_formato_yyyymm(df_nomina['MES'])
    elif 'FECHA_NOMINA' in df_nomina.columns: 
        df_nomina['FECHA_NOMINA'] = pd.to_datetime(df_nomina['FECHA_NOMINA'], errors='coerce')
        df_nomina['MES'] = df_nomina['FECHA_NOMINA'].dt.strftime('%Y-%m')
    else: df_nomina['MES'] = 'Desconocido'
    
    for col in columnas_nomina_sumar:
        df_nomina[col] = limpiar_columna_monto_alternativa(df_nomina[col]) if col in df_nomina.columns else 0
else:
    df_nomina = pd.DataFrame(columns=['MARCA', 'UBICACION', 'MES'] + columnas_nomina_sumar)

# --- Procesamiento de Alquileres (Ahora usa el df combinado de todas las hojas) ---
df_alquiler_agrupado = pd.DataFrame() 
target_monto_col_alquiler = 'MONTO BASE LIMPIO' 
if "CONTROL ALQUILER.xlsx" in dataframes and dataframes["CONTROL ALQUILER.xlsx"] is not None and not dataframes["CONTROL ALQUILER.xlsx"].empty:
    df_alquiler = dataframes["CONTROL ALQUILER.xlsx"].copy() 
    print("\n--- Procesando DataFrame COMBINADO 'CONTROL ALQUILER.xlsx' ---")
    
    marca_col_alquiler = next((col for col in ['MARCA', 'Marca'] if col in df_alquiler.columns), None)
    if marca_col_alquiler:
        if marca_col_alquiler != 'Marca': df_alquiler.rename(columns={marca_col_alquiler: 'Marca'}, inplace=True)
        df_alquiler['Marca'] = df_alquiler['Marca'].fillna('DESCONOCIDA').astype(str).str.upper().str.strip()
        print(f"DEBUG ALQUILER: Marca después de fillna/upper/strip: {df_alquiler['Marca'].unique()[:10]}") 
    else: df_alquiler['Marca'] = 'DESCONOCIDA'

    ubicacion_col_alquiler = next((col for col in ['UBICACION', 'Ubicacion', 'UBICACION_ALQUILER'] if col in df_alquiler.columns), None)
    if ubicacion_col_alquiler:
        if ubicacion_col_alquiler != 'UBICACION': df_alquiler.rename(columns={ubicacion_col_alquiler: 'UBICACION'}, inplace=True)
        df_alquiler['UBICACION'] = df_alquiler['UBICACION'].fillna('DESCONOCIDA').astype(str).str.upper().str.strip()
    else: df_alquiler['UBICACION'] = 'DESCONOCIDA'

    mes_col_alquiler = next((col for col in ['MES', 'Mes'] if col in df_alquiler.columns), None)
    if mes_col_alquiler:
        if mes_col_alquiler != 'MES': df_alquiler.rename(columns={mes_col_alquiler: 'MES'}, inplace=True)
        df_alquiler['MES'] = convertir_nombre_mes_a_formato_yyyymm(df_alquiler['MES'])
    else: df_alquiler['MES'] = 'Desconocido'

    monto_col_candidatas_alquiler = ['MONTO BASE', 'Monto Base', 'MONTOBASE', 'monto base', 'MontoBase', 'ALQUILER', 'Alquiler']
    monto_col_alquiler_encontrada = next((col for col in monto_col_candidatas_alquiler if col in df_alquiler.columns), None)
    if monto_col_alquiler_encontrada:
        df_alquiler[target_monto_col_alquiler] = limpiar_columna_monto_alternativa(df_alquiler[monto_col_alquiler_encontrada])
    else: df_alquiler[target_monto_col_alquiler] = 0

    required_alquiler_cols_group = ['Marca', 'UBICACION', 'MES', target_monto_col_alquiler]
    if all(col in df_alquiler.columns for col in required_alquiler_cols_group):
        try:
            df_alquiler.dropna(subset=['Marca', 'UBICACION', 'MES'], inplace=True) 
            df_alquiler_agrupado = df_alquiler.groupby(['Marca', 'UBICACION', 'MES']).agg(
                Monto_Alquiler_Total=(target_monto_col_alquiler, 'sum')
            ).reset_index()
            if not df_alquiler_agrupado.empty:
                print(f"DEBUG ALQUILER (Combinado): df_alquiler_agrupado['Monto_Alquiler_Total'].sum(): {df_alquiler_agrupado['Monto_Alquiler_Total'].sum()}")
                print(f"DEBUG ALQUILER (Combinado): Marcas únicas en agrupado: {df_alquiler_agrupado['Marca'].unique()}") 
        except Exception as e: print(f"Error al agrupar alquileres combinados: {e}")
    else: print(f"Advertencia ALQUILER (Combinado): Columnas para agrupar no encontradas.")
else: print("Advertencia: DataFrame 'CONTROL ALQUILER.xlsx' no encontrado o vacío en diccionario 'dataframes'.")
# Asegurar que df_alquiler_agrupado existe aunque sea vacío
if df_alquiler_agrupado.empty:
    df_alquiler_agrupado = pd.DataFrame(columns=['Marca', 'UBICACION', 'MES', 'Monto_Alquiler_Total'])


# --- Procesamiento de Condominios ---
df_condominio_agrupado = pd.DataFrame()
if 'CONDOMINIOS.xlsx' in dataframes and isinstance(dataframes['CONDOMINIOS.xlsx'], pd.DataFrame) and not dataframes['CONDOMINIOS.xlsx'].empty:
    df_condominio = dataframes['CONDOMINIOS.xlsx'].copy()
    
    marca_col_condominio = next((col for col in ['MARCA', 'Marca'] if col in df_condominio.columns), None)
    if marca_col_condominio:
        if marca_col_condominio != 'Marca': df_condominio.rename(columns={marca_col_condominio: 'Marca'}, inplace=True)
        df_condominio['Marca'] = df_condominio['Marca'].fillna('DESCONOCIDA').astype(str).str.upper().str.strip()
    else: df_condominio['Marca'] = 'DESCONOCIDA'
    
    ubicacion_col_condominio = next((col for col in ['UBICACION', 'Ubicacion'] if col in df_condominio.columns), None)
    if ubicacion_col_condominio:
        if ubicacion_col_condominio != 'UBICACION': df_condominio.rename(columns={ubicacion_col_condominio: 'UBICACION'}, inplace=True)
        df_condominio['UBICACION'] = df_condominio['UBICACION'].fillna('DESCONOCIDA').astype(str).str.upper().str.strip()
    else: df_condominio['UBICACION'] = 'DESCONOCIDA'

    if 'MES' in df_condominio.columns: df_condominio['MES'] = convertir_nombre_mes_a_formato_yyyymm(df_condominio['MES'])
    else: df_condominio['MES'] = 'Desconocido'
    
    monto_col_condominio = 'Monto $'
    df_condominio[monto_col_condominio] = limpiar_columna_monto_alternativa(df_condominio[monto_col_condominio]) if monto_col_condominio in df_condominio.columns else 0
    
    required_condominio_cols = ['Marca', 'UBICACION', 'MES', monto_col_condominio]
    if all(col in df_condominio.columns for col in required_condominio_cols):
        try:
            df_condominio.dropna(subset=['Marca', 'UBICACION', 'MES'], inplace=True)
            df_condominio_agrupado = df_condominio.groupby(['Marca', 'UBICACION', 'MES']).agg(
                Monto_Condominio_Total=(monto_col_condominio, 'sum')
            ).reset_index()
        except Exception as e: print(f"Error al agrupar condominios: {e}")
    else: print("Advertencia CONDOMINIOS: Columnas para agrupar no encontradas.")
else: print("Advertencia: 'CONDOMINIOS.xlsx' no encontrado o vacío.")
# Asegurar que df_condominio_agrupado existe aunque sea vacío
if df_condominio_agrupado.empty:
    df_condominio_agrupado = pd.DataFrame(columns=['Marca', 'UBICACION', 'MES', 'Monto_Condominio_Total'])

# --- Procesamiento de Alcaldias ---
df_alcaldias_agrupado = pd.DataFrame()
if 'ALCALDIAS.xlsx' in dataframes and dataframes['ALCALDIAS.xlsx'] is not None and not dataframes['ALCALDIAS.xlsx'].empty:
    df_alcaldias_original = dataframes['ALCALDIAS.xlsx'].copy()
    
    marca_col_alcaldias = next((col for col in ['MARCA', 'Marca'] if col in df_alcaldias_original.columns), None)
    if marca_col_alcaldias:
        if marca_col_alcaldias != 'Marca': df_alcaldias_original.rename(columns={marca_col_alcaldias: 'Marca'}, inplace=True)
        df_alcaldias_original['Marca'] = df_alcaldias_original['Marca'].fillna('DESCONOCIDA').astype(str).str.upper().str.strip()
    else: df_alcaldias_original['Marca'] = 'DESCONOCIDA'

    if 'MES' in df_alcaldias_original.columns:
        df_alcaldias_original['MES'] = convertir_nombre_mes_a_formato_yyyymm(df_alcaldias_original['MES'])
    elif 'FECHA_DECLARACION' in df_alcaldias_original.columns:
        df_alcaldias_original['FECHA_DECLARACION'] = pd.to_datetime(df_alcaldias_original['FECHA_DECLARACION'], errors='coerce')
        df_alcaldias_original['MES'] = df_alcaldias_original['FECHA_DECLARACION'].dt.strftime('%Y-%m')
    else: df_alcaldias_original['MES'] = 'Desconocido'

    monto_col_alcaldias = 'MONTO $'
    df_alcaldias_original[monto_col_alcaldias] = limpiar_columna_monto_alternativa(df_alcaldias_original[monto_col_alcaldias]) if monto_col_alcaldias in df_alcaldias_original.columns else 0
    
    required_alcaldias_cols = ['Marca', 'MES', monto_col_alcaldias]
    if all(col in df_alcaldias_original.columns for col in required_alcaldias_cols):
        df_alcaldias_original.dropna(subset=['Marca', 'MES'], inplace=True)
        df_total_impuestos_alcaldia_marca_mes = df_alcaldias_original.groupby(['Marca', 'MES']).agg(
            Total_Impuesto_Alcaldia_Marca_Mes=(monto_col_alcaldias, 'sum')
        ).reset_index()
        if not df_ventas.empty and all(c in df_ventas.columns for c in ['Marca', 'UBICACION', 'MES', 'Ventas $']):
            if 'UBICACION' in df_ventas.columns:
                df_ventas_cleaned = df_ventas.dropna(subset=['Marca', 'UBICACION', 'MES'])
                df_ventas_detalle_mes = df_ventas_cleaned.groupby(['Marca', 'UBICACION', 'MES']).agg(Ventas_Ubicacion_Mes=('Ventas $', 'sum')).reset_index()
                df_ventas_total_marca_mes = df_ventas_detalle_mes.groupby(['Marca', 'MES']).agg(Ventas_Total_Marca_Mes=('Ventas_Ubicacion_Mes', 'sum')).reset_index()
                df_ventas_con_total_marca_mes = pd.merge(df_ventas_detalle_mes, df_ventas_total_marca_mes, on=['Marca', 'MES'], how='left')
                df_ventas_con_total_marca_mes['Pct_Ventas_Ubicacion'] = 0.0
                mask_ventas_validas = (df_ventas_con_total_marca_mes['Ventas_Total_Marca_Mes'] != 0) & (df_ventas_con_total_marca_mes['Ventas_Total_Marca_Mes'].notna())
                df_ventas_con_total_marca_mes.loc[mask_ventas_validas, 'Pct_Ventas_Ubicacion'] = (df_ventas_con_total_marca_mes.loc[mask_ventas_validas, 'Ventas_Ubicacion_Mes'] / df_ventas_con_total_marca_mes.loc[mask_ventas_validas, 'Ventas_Total_Marca_Mes'])
                df_ventas_con_total_marca_mes['Pct_Ventas_Ubicacion'].fillna(0, inplace=True)
                df_distribucion_alcaldias = pd.merge(df_ventas_con_total_marca_mes, df_total_impuestos_alcaldia_marca_mes, on=['Marca', 'MES'], how='left')
                df_distribucion_alcaldias['Total_Impuesto_Alcaldia_Marca_Mes'].fillna(0, inplace=True)
                df_distribucion_alcaldias['Monto_Alcaldia_Total'] = df_distribucion_alcaldias['Pct_Ventas_Ubicacion'] * df_distribucion_alcaldias['Total_Impuesto_Alcaldia_Marca_Mes']
                if all(col in df_distribucion_alcaldias.columns for col in ['Marca', 'UBICACION', 'MES', 'Monto_Alcaldia_Total']):
                     df_alcaldias_agrupado = df_distribucion_alcaldias[['Marca', 'UBICACION', 'MES', 'Monto_Alcaldia_Total']].copy()
            else:
                print("Advertencia ALCALDIAS: Columna 'UBICACION' no encontrada en df_ventas para distribución.")
    else: print(f"Advertencia ALCALDIAS: Columnas para agrupar no encontradas.")
else: print("Advertencia: DataFrame 'ALCALDIAS.xlsx' no encontrado o vacío.")
if df_alcaldias_agrupado.empty: 
    df_alcaldias_agrupado = pd.DataFrame(columns=['Marca', 'UBICACION', 'MES', 'Monto_Alcaldia_Total'])

# --- Cálculo de Ventas por Marca y Ubicación (para gráficos) ---
df_ventas_agrupado = pd.DataFrame()
if not df_ventas.empty and all(col in df_ventas.columns for col in ['Marca', 'UBICACION', 'Ventas $', 'Coste $']):
    try:
        df_temp_ventas = df_ventas.copy() 
        df_temp_ventas['Ventas $'] = pd.to_numeric(df_temp_ventas['Ventas $'], errors='coerce').fillna(0)
        df_temp_ventas['Coste $'] = pd.to_numeric(df_temp_ventas['Coste $'], errors='coerce').fillna(0)
        df_temp_ventas.dropna(subset=['Marca', 'UBICACION'], inplace=True)
        df_ventas_agrupado = df_temp_ventas.groupby(['Marca', 'UBICACION']).agg(
            Ventas_Marca_Ubicacion=('Ventas $', 'sum'),
            Coste_Ventas_Marca_Ubicacion=('Coste $', 'sum')
        ).reset_index()
        df_ventas_agrupado['Margen_Bruto_Marca_Ubicacion'] = df_ventas_agrupado['Ventas_Marca_Ubicacion'] - df_ventas_agrupado['Coste_Ventas_Marca_Ubicacion']
    except Exception as e: print(f"Error al calcular ventas agrupadas: {e}")
if df_ventas_agrupado.empty:
    df_ventas_agrupado = pd.DataFrame(columns=['Marca', 'UBICACION', 'Ventas_Marca_Ubicacion', 'Coste_Ventas_Marca_Ubicacion', 'Margen_Bruto_Marca_Ubicacion'])

# --- Sumatoria de Nómina por Marca y Ubicación (para gráficos) ---
df_nomina_agrupado = pd.DataFrame()
if not df_nomina.empty and all(col in df_nomina.columns for col in ['MARCA', 'UBICACION']):
    try:
        df_temp_nomina = df_nomina.copy()
        df_temp_nomina.dropna(subset=['MARCA', 'UBICACION'], inplace=True)
        cols_existentes_nomina = [col for col in columnas_nomina_sumar if col in df_temp_nomina.columns]
        if cols_existentes_nomina:
             for col in cols_existentes_nomina:
                  df_temp_nomina[col] = pd.to_numeric(df_temp_nomina[col], errors='coerce').fillna(0)
             df_nomina_agrupado_temp = df_temp_nomina.groupby(['MARCA', 'UBICACION'])[cols_existentes_nomina].sum().reset_index()
             df_nomina_agrupado_temp['Nomina_Marca_Ubicacion'] = df_nomina_agrupado_temp[cols_existentes_nomina].sum(axis=1)
             df_nomina_agrupado = df_nomina_agrupado_temp.rename(columns={'MARCA': 'Marca'})[['Marca', 'UBICACION', 'Nomina_Marca_Ubicacion']]
    except Exception as e: print(f"Error al calcular nómina agrupada: {e}")
if df_nomina_agrupado.empty:
    df_nomina_agrupado = pd.DataFrame(columns=['Marca', 'UBICACION', 'Nomina_Marca_Ubicacion'])

# 4. Diseño del Layout de la Aplicación
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Estado de Ganancias y Pérdidas"

# Asegurar que df_ventas_agrupado tiene 'Marca' antes de generar opciones
if 'Marca' in df_ventas_agrupado.columns and not df_ventas_agrupado.empty:
    unique_marcas = sorted([m for m in df_ventas_agrupado['Marca'].unique() if m != 'DESCONOCIDA'])
    initial_marca_options = [{'label': marca, 'value': marca} for marca in unique_marcas]
    initial_selected_marca = unique_marcas[0] if unique_marcas else None
else:
    initial_marca_options = []
    initial_selected_marca = None

app.layout = dbc.Container([
    dbc.Row(dbc.Col(html.H1("Estado de Ganancias y Pérdidas", className="text-center my-4"))),
    dbc.Row([
        dbc.Col([
            html.Label("Marca:", className="form-label"),
            dcc.Dropdown(id='marca-dropdown', options=initial_marca_options, value=initial_selected_marca, clearable=False)
        ], width=12, sm=6, md=4, lg=3, className="mb-2"),
        dbc.Col([
            html.Label("Ubicación:", className="form-label"),
            dcc.Dropdown(id='ubicacion-dropdown', options=[], value=None, clearable=True, placeholder="Todas las ubicaciones")
        ], width=12, sm=6, md=4, lg=3, className="mb-2"),
        dbc.Col([
            html.Label("Mes:", className="form-label"),
            dcc.Dropdown(id='mes-dropdown', options=[], value=None, clearable=True, placeholder="Todos los meses")
        ], width=12, sm=12, md=4, lg=3, className="mb-2")
    ], className="mb-4 justify-content-center"),
    dbc.Row([
        dbc.Col([html.H2("Indicadores P&L Clave", className="text-center my-3"), html.Div(id='pnl-summary-container')], width=12, lg=6, className="mb-4"),
        dbc.Col([html.H2("Margen vs Ventas por Marca (Total)", className="text-center my-3"), dcc.Graph(id='scatter-marca-total')], width=12, lg=6, className="mb-4"),
    ], className="mb-4"),
    dbc.Row([
        dbc.Col(dcc.Graph(id='nomina-bar-chart'), width=12, lg=6, className="mb-4"),
        dbc.Col(dcc.Graph(id='ventas-bar-chart'), width=12, lg=6, className="mb-4")
    ]),
    dbc.Row(dbc.Col(dcc.Graph(id='scatter-chart'), width=12, className="mb-4")),
    dbc.Row(dbc.Col([
        html.H3("Detalle P&L por Ubicación (para Marca Seleccionada)", className="text-center my-3"),
        dash_table.DataTable(
            id='tabla-pnl-razon-social',
            style_table={'overflowX': 'auto', 'width': '100%', 'minWidth': '100%'},
            style_cell={'textAlign': 'left', 'padding': '8px', 'fontFamily': 'Arial, sans-serif', 'minWidth': '120px', 'width': '150px', 'maxWidth': '200px', 'whiteSpace': 'normal', 'height': 'auto'},
            style_header={'backgroundColor': '#f0f0f0', 'fontWeight': 'bold', 'border': '1px solid #dee2e6', 'padding': '10px'},
            style_data_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}],
            style_data={'border': '1px solid #dee2e6', 'padding': '8px'},
            page_size=10,
        )
    ], width=12, md=10, className="mx-auto"))
], fluid=True, className="p-3")

# 5. Callbacks
@app.callback(
    [Output('ubicacion-dropdown', 'options'), Output('ubicacion-dropdown', 'value')],
    [Input('marca-dropdown', 'value')],
    [State('ubicacion-dropdown', 'value')]
)
def actualizar_dropdown_ubicacion(marca_seleccionada, ubicacion_actual_seleccionada):
    if not marca_seleccionada: return [], None
    ubicaciones_disponibles_set = set()
    datasets_para_ubicaciones = {
        "ventas": (df_ventas, 'Marca', 'UBICACION'), 
        "nomina": (df_nomina, 'MARCA', 'UBICACION'),
        "alquiler": (df_alquiler_agrupado, 'Marca', 'UBICACION'), 
        "condominio": (df_condominio_agrupado, 'Marca', 'UBICACION'),
        "alcaldias": (df_alcaldias_agrupado, 'Marca', 'UBICACION')
    }
    marca_sel_std = str(marca_seleccionada).upper().strip() 
    for name, (df, marca_col, ubicacion_col) in datasets_para_ubicaciones.items():
        if df is not None and not df.empty and marca_col in df.columns and ubicacion_col in df.columns:
             df_temp = df[df[marca_col].astype(str).str.upper().str.strip() == marca_sel_std]
             ubicaciones_disponibles_set.update(df_temp[ubicacion_col].unique())
    
    ubicaciones_disponibles_set = {str(u).upper().strip() for u in ubicaciones_disponibles_set if pd.notna(u) and str(u).strip() != '' and str(u).upper().strip() != 'DESCONOCIDA' and str(u).upper().strip() != 'DESCONOCIDO'}
    ubicaciones_disponibles = sorted(list(ubicaciones_disponibles_set))
    opciones_ubicacion = [{'label': ubicacion, 'value': ubicacion} for ubicacion in ubicaciones_disponibles]
    
    current_value = None
    if ubicacion_actual_seleccionada and str(ubicacion_actual_seleccionada).upper().strip() in ubicaciones_disponibles:
        current_value = str(ubicacion_actual_seleccionada).upper().strip()
    
    return opciones_ubicacion, current_value

@app.callback(
    Output('mes-dropdown', 'options'),
    [Input('marca-dropdown', 'value'), Input('ubicacion-dropdown', 'value')]
)
def actualizar_dropdown_mes(marca_seleccionada, ubicacion_seleccionada):
    if not marca_seleccionada: return []
    meses_disponibles_set = set()
    datasets_para_mes = {
        "ventas": (df_ventas, 'Marca', 'UBICACION', 'MES'), 
        "nomina": (df_nomina, 'MARCA', 'UBICACION', 'MES'),
        "alquiler": (df_alquiler_agrupado, 'Marca', 'UBICACION', 'MES'),
        "condominio": (df_condominio_agrupado, 'Marca', 'UBICACION', 'MES'),
        "alcaldias": (df_alcaldias_agrupado, 'Marca', 'UBICACION', 'MES')
    }
    marca_sel_std = str(marca_seleccionada).upper().strip()
    ubicacion_sel_std = str(ubicacion_seleccionada).upper().strip() if ubicacion_seleccionada else None

    for nombre_df, (df_iter, marca_col, ubicacion_col, mes_col) in datasets_para_mes.items():
        if df_iter is None or df_iter.empty or mes_col not in df_iter.columns: continue
        df_filtrado_mes = df_iter.copy()
        if marca_sel_std and marca_col in df_filtrado_mes.columns:
             df_filtrado_mes = df_filtrado_mes[df_filtrado_mes[marca_col].astype(str).str.upper().str.strip() == marca_sel_std]
        
        if ubicacion_sel_std and ubicacion_col in df_filtrado_mes.columns:
             df_filtrado_mes = df_filtrado_mes[df_filtrado_mes[ubicacion_col].astype(str).str.upper().str.strip() == ubicacion_sel_std]
        
        if not df_filtrado_mes.empty and mes_col in df_filtrado_mes.columns:
            meses_disponibles_set.update(df_filtrado_mes[mes_col].unique())
            
    meses_disponibles_set = {m for m in meses_disponibles_set if pd.notna(m) and str(m).strip() != '' and str(m).lower() != 'desconocido'}
    meses_validos = [m for m in meses_disponibles_set if isinstance(m, str) and re.match(r'^\d{4}-\d{2}$', m)]
    meses_ordenados = sorted(meses_validos)
    return [{'label': mes, 'value': mes} for mes in meses_ordenados]

@app.callback(
    [Output('pnl-summary-container', 'children'),
     Output('tabla-pnl-razon-social', 'data'),
     Output('tabla-pnl-razon-social', 'columns')],
    [Input('marca-dropdown', 'value'),
     Input('ubicacion-dropdown', 'value'),
     Input('mes-dropdown', 'value')]
)
def actualizar_indicadores_pnl_y_tabla(marca_seleccionada, ubicacion_seleccionada, mes_seleccionado):
    print(f"\n--- DEBUG CALLBACK P&L ---")
    print(f"Filtros: Marca='{marca_seleccionada}', Ubicación='{ubicacion_seleccionada}', Mes='{mes_seleccionado}'")

    # Estandarizar filtros de entrada
    marca_sel_std = str(marca_seleccionada).upper().strip() if marca_seleccionada else None
    ubicacion_sel_std = str(ubicacion_seleccionada).upper().strip() if ubicacion_seleccionada else None
    mes_sel_std = str(mes_seleccionado).strip() if mes_seleccionado else None 

    ventas_total, costo_ventas_total, margen_bruto_total, nomina_total = 0.0, 0.0, 0.0, 0.0
    alquiler_total_indicadores, condominio_total_indicadores, alcaldias_total_indicadores = 0.0, 0.0, 0.0

    # Crear copias para filtrado local
    df_venta_filt = df_ventas.copy() if df_ventas is not None and not df_ventas.empty else pd.DataFrame(columns=df_ventas.columns if hasattr(df_ventas, 'columns') else [])
    df_nomina_filt = df_nomina.copy() if df_nomina is not None and not df_nomina.empty else pd.DataFrame(columns=df_nomina.columns if hasattr(df_nomina, 'columns') else [])
    df_alquiler_filt = df_alquiler_agrupado.copy() if df_alquiler_agrupado is not None and not df_alquiler_agrupado.empty else pd.DataFrame(columns=df_alquiler_agrupado.columns if hasattr(df_alquiler_agrupado, 'columns') else [])
    df_condominio_filt = df_condominio_agrupado.copy() if df_condominio_agrupado is not None and not df_condominio_agrupado.empty else pd.DataFrame(columns=df_condominio_agrupado.columns if hasattr(df_condominio_agrupado, 'columns') else [])
    df_alcaldias_filt = df_alcaldias_agrupado.copy() if df_alcaldias_agrupado is not None and not df_alcaldias_agrupado.empty else pd.DataFrame(columns=df_alcaldias_agrupado.columns if hasattr(df_alcaldias_agrupado, 'columns') else [])

    # Aplicar filtros para indicadores
    if marca_sel_std:
        if 'Marca' in df_venta_filt.columns: df_venta_filt = df_venta_filt[df_venta_filt['Marca'] == marca_sel_std]
        if 'MARCA' in df_nomina_filt.columns: df_nomina_filt = df_nomina_filt[df_nomina_filt['MARCA'] == marca_sel_std] 
        if 'Marca' in df_alquiler_filt.columns: df_alquiler_filt = df_alquiler_filt[df_alquiler_filt['Marca'] == marca_sel_std]
        if 'Marca' in df_condominio_filt.columns: df_condominio_filt = df_condominio_filt[df_condominio_filt['Marca'] == marca_sel_std]
        if 'Marca' in df_alcaldias_filt.columns: df_alcaldias_filt = df_alcaldias_filt[df_alcaldias_filt['Marca'] == marca_sel_std]
    
    if ubicacion_sel_std:
        if 'UBICACION' in df_venta_filt.columns: df_venta_filt = df_venta_filt[df_venta_filt['UBICACION'] == ubicacion_sel_std]
        if 'UBICACION' in df_nomina_filt.columns: df_nomina_filt = df_nomina_filt[df_nomina_filt['UBICACION'] == ubicacion_sel_std]
        if 'UBICACION' in df_alquiler_filt.columns: df_alquiler_filt = df_alquiler_filt[df_alquiler_filt['UBICACION'] == ubicacion_sel_std]
        if 'UBICACION' in df_condominio_filt.columns: df_condominio_filt = df_condominio_filt[df_condominio_filt['UBICACION'] == ubicacion_sel_std]
        if 'UBICACION' in df_alcaldias_filt.columns: df_alcaldias_filt = df_alcaldias_filt[df_alcaldias_filt['UBICACION'] == ubicacion_sel_std]

    if mes_sel_std:
        if 'MES' in df_venta_filt.columns: df_venta_filt = df_venta_filt[df_venta_filt['MES'] == mes_sel_std]
        if 'MES' in df_nomina_filt.columns: df_nomina_filt = df_nomina_filt[df_nomina_filt['MES'] == mes_sel_std]
        if 'MES' in df_alquiler_filt.columns: df_alquiler_filt = df_alquiler_filt[df_alquiler_filt['MES'] == mes_sel_std]
        if 'MES' in df_condominio_filt.columns: df_condominio_filt = df_condominio_filt[df_condominio_filt['MES'] == mes_sel_std]
        if 'MES' in df_alcaldias_filt.columns: df_alcaldias_filt = df_alcaldias_filt[df_alcaldias_filt['MES'] == mes_sel_std]

    # **DEBUG ADICIONAL**: Mostrar datos de alquiler *después* de aplicar filtros
    print(f"DEBUG P&L Indicadores DESPUÉS FILTRO: df_alquiler_filt (shape: {df_alquiler_filt.shape})")
    if not df_alquiler_filt.empty:
        print(f"  Suma Monto_Alquiler_Total: {df_alquiler_filt['Monto_Alquiler_Total'].sum() if 'Monto_Alquiler_Total' in df_alquiler_filt.columns else 'Columna no encontrada'}")
        print(f"  Marcas únicas en df_alquiler_filt: {df_alquiler_filt['Marca'].unique() if 'Marca' in df_alquiler_filt.columns else 'Columna no encontrada'}")
        print(f"  Ubicaciones únicas en df_alquiler_filt: {df_alquiler_filt['UBICACION'].unique() if 'UBICACION' in df_alquiler_filt.columns else 'Columna no encontrada'}")

    # Calcular totales para indicadores
    if not df_venta_filt.empty:
        if 'Ventas $' in df_venta_filt.columns: ventas_total = df_venta_filt['Ventas $'].sum()
        if 'Coste $' in df_venta_filt.columns: costo_ventas_total = df_venta_filt['Coste $'].sum()
    margen_bruto_total = ventas_total - costo_ventas_total

    if not df_nomina_filt.empty:
        cols_existentes_nomina = [col for col in columnas_nomina_sumar if col in df_nomina_filt.columns]
        if cols_existentes_nomina: nomina_total = df_nomina_filt[cols_existentes_nomina].sum(axis=1).sum()
    
    if not df_alquiler_filt.empty and 'Monto_Alquiler_Total' in df_alquiler_filt.columns:
        alquiler_total_indicadores = df_alquiler_filt['Monto_Alquiler_Total'].sum()
    if not df_condominio_filt.empty and 'Monto_Condominio_Total' in df_condominio_filt.columns:
        condominio_total_indicadores = df_condominio_filt['Monto_Condominio_Total'].sum()
    if not df_alcaldias_filt.empty and 'Monto_Alcaldia_Total' in df_alcaldias_filt.columns:
        alcaldias_total_indicadores = df_alcaldias_filt['Monto_Alcaldia_Total'].sum()
    
    # Calcular porcentajes
    pct_costo_ventas = (costo_ventas_total / ventas_total * 100) if ventas_total else 0
    pct_margen_bruto = (margen_bruto_total / ventas_total * 100) if ventas_total else 0
    pct_nomina = (nomina_total / ventas_total * 100) if ventas_total else 0
    pct_alquiler = (alquiler_total_indicadores / ventas_total * 100) if ventas_total else 0
    pct_condominio = (condominio_total_indicadores / ventas_total * 100) if ventas_total else 0
    ebitda_tienda_indicadores = margen_bruto_total - nomina_total - alquiler_total_indicadores - condominio_total_indicadores
    pct_ebitda_tienda = (ebitda_tienda_indicadores / ventas_total * 100) if ventas_total else 0
    pct_alcaldias = (alcaldias_total_indicadores / ventas_total * 100) if ventas_total else 0
    ebitda_final_indicadores = ebitda_tienda_indicadores - alcaldias_total_indicadores
    pct_ebitda_final = (ebitda_final_indicadores / ventas_total * 100) if ventas_total else 0

    # Construir tabla resumen
    pnl_summary_card_content = [
        html.Thead(html.Tr([html.Th("Indicador"), html.Th("Monto ($)", style={'textAlign': 'right'}), html.Th("% Ventas", style={'textAlign': 'right'})])),
        html.Tbody([
            html.Tr([html.Td("Ventas Netas"), html.Td(f"{ventas_total:,.2f}", style={'textAlign': 'right'}), html.Td(f"{100.00:,.2f}%" if ventas_total else "0.00%", style={'textAlign': 'right'})]),
            html.Tr([html.Td("Costo de Ventas"), html.Td(f"{costo_ventas_total:,.2f}", style={'textAlign': 'right'}), html.Td(f"{pct_costo_ventas:,.2f}%", style={'textAlign': 'right'})]),
            html.Tr([html.Td("Margen Bruto", style={'fontWeight': 'bold'}), html.Td(f"{margen_bruto_total:,.2f}", style={'fontWeight': 'bold', 'textAlign': 'right'}), html.Td(f"{pct_margen_bruto:,.2f}%", style={'fontWeight': 'bold', 'textAlign': 'right'})]),
            html.Tr([html.Td("Gastos de Nómina"), html.Td(f"{nomina_total:,.2f}", style={'textAlign': 'right'}), html.Td(f"{pct_nomina:,.2f}%", style={'textAlign': 'right'})]),
            html.Tr([html.Td("Gastos de Alquiler"), html.Td(f"{alquiler_total_indicadores:,.2f}", style={'textAlign': 'right'}), html.Td(f"{pct_alquiler:,.2f}%", style={'textAlign': 'right'})]),
            html.Tr([html.Td("Gastos de Condominio"), html.Td(f"{condominio_total_indicadores:,.2f}", style={'textAlign': 'right'}), html.Td(f"{pct_condominio:,.2f}%", style={'textAlign': 'right'})]),
            html.Tr([html.Td("EBITDA Tienda", style={'fontWeight': 'bold'}), html.Td(f"{ebitda_tienda_indicadores:,.2f}", style={'fontWeight': 'bold', 'textAlign': 'right'}), html.Td(f"{pct_ebitda_tienda:,.2f}%", style={'fontWeight': 'bold', 'textAlign': 'right'})]),
            html.Tr([html.Td("Impuestos Alcaldía"), html.Td(f"{alcaldias_total_indicadores:,.2f}", style={'textAlign': 'right'}), html.Td(f"{pct_alcaldias:,.2f}%", style={'textAlign': 'right'})]),
            html.Tr([html.Td("EBITDA", style={'fontWeight': 'bold', 'borderTop': '1px solid #ccc'}), html.Td(f"{ebitda_final_indicadores:,.2f}", style={'fontWeight': 'bold', 'textAlign': 'right', 'borderTop': '1px solid #ccc'}), html.Td(f"{pct_ebitda_final:,.2f}%", style={'fontWeight': 'bold', 'textAlign': 'right', 'borderTop': '1px solid #ccc'})]),
        ])]
    pnl_summary_table = dbc.Table(pnl_summary_card_content, bordered=False, striped=True, hover=True, responsive=True, className="mb-0 table-sm")
    cards_component = dbc.Card(dbc.CardBody([html.H4("Resumen P&L", className="card-title text-center mb-3"), pnl_summary_table]), className="shadow-sm")

    # --- Preparación de la tabla P&L Detallada por Ubicación ---
    tabla_data, tabla_columnas = [], []
    if marca_sel_std:
        # Obtener ubicaciones únicas para la marca seleccionada (usando dataframes originales)
        ubicaciones_marca_set = set()
        datasets_para_tabla_ubicaciones = {
            "ventas": (df_ventas, 'Marca', 'UBICACION'), 
            "nomina": (df_nomina, 'MARCA', 'UBICACION'),
            "alquiler": (df_alquiler_agrupado, 'Marca', 'UBICACION'), 
            "condominio": (df_condominio_agrupado, 'Marca', 'UBICACION'),
            "alcaldias": (df_alcaldias_agrupado, 'Marca', 'UBICACION')
        }
        for name, (df_iter_tabla, mc_tabla, uc_tabla) in datasets_para_tabla_ubicaciones.items():
            if df_iter_tabla is not None and not df_iter_tabla.empty and mc_tabla in df_iter_tabla.columns and uc_tabla in df_iter_tabla.columns:
                mask_marca = df_iter_tabla[mc_tabla].astype(str).str.upper().str.strip() == marca_sel_std
                if mask_marca.any():
                    df_temp_ubicaciones_tabla = df_iter_tabla[mask_marca]
                    ubicaciones_marca_set.update(df_temp_ubicaciones_tabla[uc_tabla].unique())
        
        ubicaciones_marca_limpias = sorted(list({str(u).upper().strip() for u in ubicaciones_marca_set if pd.notna(u) and str(u).strip() != '' and str(u).upper().strip() != 'DESCONOCIDA'}))

        if not ubicaciones_marca_limpias:
            return cards_component, [{'Mensaje': f"No hay ubicaciones válidas para la marca '{marca_sel_std}'."}], [{"name": "Mensaje", "id": "Mensaje"}]

        df_tabla_pnl_final = pd.DataFrame(list(ubicaciones_marca_limpias), columns=['UBICACION'])
        
        # Usar los dataframes ya filtrados (df_venta_filt, df_nomina_filt, etc.)
        # Agrupar estos dataframes filtrados por UBICACION para la tabla.
        
        # Ventas para la tabla
        if not df_venta_filt.empty and all(c in df_venta_filt for c in ['UBICACION', 'Ventas $', 'Coste $']):
            df_v = df_venta_filt.groupby('UBICACION', as_index=False).agg(Ventas_Ubicacion=('Ventas $', 'sum'), Costo_Ventas_Ubicacion=('Coste $', 'sum'))
            if not df_v.empty: df_v['Margen_Bruto_Ubicacion'] = df_v['Ventas_Ubicacion'] - df_v['Costo_Ventas_Ubicacion']
            df_tabla_pnl_final = pd.merge(df_tabla_pnl_final, df_v, on='UBICACION', how='left')
        
        # Nómina para la tabla
        if not df_nomina_filt.empty and 'UBICACION' in df_nomina_filt.columns:
            cols_n_tabla = [col for col in columnas_nomina_sumar if col in df_nomina_filt.columns]
            if cols_n_tabla:
                df_n = df_nomina_filt.groupby('UBICACION', as_index=False)[cols_n_tabla].sum()
                if not df_n.empty: df_n['Nomina_Marca_Ubicacion'] = df_n[cols_n_tabla].sum(axis=1)
                df_tabla_pnl_final = pd.merge(df_tabla_pnl_final, df_n[['UBICACION', 'Nomina_Marca_Ubicacion']], on='UBICACION', how='left')

        # Alquiler para la tabla
        if not df_alquiler_filt.empty and all(c in df_alquiler_filt for c in ['UBICACION', 'Monto_Alquiler_Total']):
            df_a = df_alquiler_filt.groupby('UBICACION', as_index=False)['Monto_Alquiler_Total'].sum()
            df_tabla_pnl_final = pd.merge(df_tabla_pnl_final, df_a, on='UBICACION', how='left')

        # Condominio para la tabla
        if not df_condominio_filt.empty and all(c in df_condominio_filt for c in ['UBICACION', 'Monto_Condominio_Total']):
            df_c = df_condominio_filt.groupby('UBICACION', as_index=False)['Monto_Condominio_Total'].sum()
            df_tabla_pnl_final = pd.merge(df_tabla_pnl_final, df_c, on='UBICACION', how='left')

        # Alcaldías para la tabla
        if not df_alcaldias_filt.empty and all(c in df_alcaldias_filt for c in ['UBICACION', 'Monto_Alcaldia_Total']):
            df_alc = df_alcaldias_filt.groupby('UBICACION', as_index=False)['Monto_Alcaldia_Total'].sum()
            df_tabla_pnl_final = pd.merge(df_tabla_pnl_final, df_alc, on='UBICACION', how='left')

        # Rellenar NaNs después de todos los merges
        fill_values = {
            'Ventas_Ubicacion': 0.0, 'Costo_Ventas_Ubicacion': 0.0, 'Margen_Bruto_Ubicacion': 0.0,
            'Nomina_Marca_Ubicacion': 0.0, 'Monto_Alquiler_Total': 0.0, 
            'Monto_Condominio_Total': 0.0, 'Monto_Alcaldia_Total': 0.0
        }
        for col_name, fill_val in fill_values.items():
            if col_name not in df_tabla_pnl_final.columns:
                df_tabla_pnl_final[col_name] = fill_val 
            else:
                df_tabla_pnl_final[col_name] = pd.to_numeric(df_tabla_pnl_final[col_name], errors='coerce')
                df_tabla_pnl_final[col_name].fillna(fill_val, inplace=True)
        
        # Calcular EBITDA y formatear
        if not df_tabla_pnl_final.empty:
            df_tabla_pnl_final['EBITDA_Tienda_Ubicacion'] = (df_tabla_pnl_final['Margen_Bruto_Ubicacion'] - 
                                                            df_tabla_pnl_final['Nomina_Marca_Ubicacion'] - 
                                                            df_tabla_pnl_final['Monto_Alquiler_Total'] - 
                                                            df_tabla_pnl_final['Monto_Condominio_Total'])
            df_tabla_pnl_final['EBITDA_Final_Ubicacion'] = df_tabla_pnl_final['EBITDA_Tienda_Ubicacion'] - df_tabla_pnl_final['Monto_Alcaldia_Total']
            
            df_tabla_pnl_final.rename(columns={
                'UBICACION': 'Ubicación', 'Ventas_Ubicacion': 'Ventas ($)', 
                'Costo_Ventas_Ubicacion': 'Costo Ventas ($)', 'Margen_Bruto_Ubicacion': 'Margen Bruto ($)',
                'Nomina_Marca_Ubicacion': 'Nómina ($)', 'Monto_Alquiler_Total': 'Alquiler ($)',
                'Monto_Condominio_Total': 'Condominio ($)', 'Monto_Alcaldia_Total': 'Alcaldías ($)',
                'EBITDA_Tienda_Ubicacion': 'EBITDA Tienda ($)', 'EBITDA_Final_Ubicacion': 'EBITDA Final ($)'
            }, inplace=True)
            cols_to_format = [col for col in df_tabla_pnl_final.columns if col != 'Ubicación']
            for col_format in cols_to_format:
                 if col_format in df_tabla_pnl_final.columns:
                    df_tabla_pnl_final[col_format] = pd.to_numeric(df_tabla_pnl_final[col_format], errors='coerce').fillna(0).apply(lambda x: f"{x:,.2f}")
            tabla_data = df_tabla_pnl_final.to_dict('records')
            tabla_columnas = [{"name": i, "id": i} for i in df_tabla_pnl_final.columns]
        else:
             tabla_data = [{'Mensaje': "No hay datos detallados para los filtros."}]
             tabla_columnas = [{"name": "Mensaje", "id": "Mensaje"}]
    else:
        tabla_data = [{'Mensaje': "Seleccione una marca para ver detalles."}]
        tabla_columnas = [{"name": "Mensaje", "id": "Mensaje"}]
    return cards_component, tabla_data, tabla_columnas

@app.callback(Output('scatter-marca-total', 'figure'), [Input('marca-dropdown', 'options')])
def actualizar_grafico_scatter_marca_total(marca_options_unused): # El input es solo para trigger
    df_plot = df_ventas.copy() if df_ventas is not None and not df_ventas.empty else pd.DataFrame()
    title_text = 'Margen vs Ventas por Marca (Total)'
    fig = go.Figure(layout={'title': f'{title_text} (No hay datos)', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}})
    
    required_cols = ['Marca', 'Ventas $', 'Coste $']
    if not df_plot.empty and all(col in df_plot.columns for col in required_cols):
          df_plot_filtered = df_plot[df_plot['Marca'] != 'DESCONOCIDA']
          if not df_plot_filtered.empty:
              df_agrupado = df_plot_filtered.groupby('Marca', as_index=False).agg(
                  Ventas_Marca=('Ventas $', 'sum'), 
                  Coste_Ventas_Marca=('Coste $', 'sum')
              )
              if not df_agrupado.empty :
                df_agrupado['Margen_Bruto_Marca'] = df_agrupado['Ventas_Marca'] - df_agrupado['Coste_Ventas_Marca']
                fig = px.scatter(df_agrupado, x='Margen_Bruto_Marca', y='Ventas_Marca', 
                                 size='Ventas_Marca', color='Marca', title=title_text, 
                                 labels={'Margen_Bruto_Marca': 'Margen Bruto Total ($)', 'Ventas_Marca': 'Ventas Totales ($)'}, 
                                 hover_name='Marca')
                fig.update_layout(title_x=0.5, plot_bgcolor='white', paper_bgcolor='white', font_color='#333', hovermode="closest")
                fig.update_traces(hovertemplate='<b>Marca</b>: %{hovertext}<br><b>Margen</b>: %{x:,.0f}<br><b>Ventas</b>: %{y:,.0f}<extra></extra>')
    return fig

@app.callback(Output('nomina-bar-chart', 'figure'), [Input('marca-dropdown', 'value'), Input('ubicacion-dropdown', 'value'), Input('mes-dropdown', 'value')])
def actualizar_grafico_nomina(marca_seleccionada, ubicacion_seleccionada, mes_seleccionado):
    df_plot = df_nomina.copy() if df_nomina is not None and not df_nomina.empty else pd.DataFrame()
    title_text = 'Nómina'
    fig = go.Figure(layout={'title': f'{title_text} (No hay datos)', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}})
    if df_plot.empty: return fig

    marca_sel_std = str(marca_seleccionada).upper().strip() if marca_seleccionada else None
    ubicacion_sel_std = str(ubicacion_seleccionada).upper().strip() if ubicacion_seleccionada else None
    mes_sel_std = str(mes_seleccionado).strip() if mes_seleccionado else None

    if marca_sel_std and 'MARCA' in df_plot.columns: df_plot = df_plot[df_plot['MARCA'] == marca_sel_std]
    if ubicacion_sel_std and 'UBICACION' in df_plot.columns: df_plot = df_plot[df_plot['UBICACION'] == ubicacion_sel_std]
    if mes_sel_std and 'MES' in df_plot.columns: df_plot = df_plot[df_plot['MES'] == mes_sel_std]
    
    if 'MARCA' in df_plot.columns: df_plot = df_plot[df_plot['MARCA'] != 'DESCONOCIDA']
    if 'UBICACION' in df_plot.columns: df_plot = df_plot[df_plot['UBICACION'] != 'DESCONOCIDA']
    
    if df_plot.empty: return fig.update_layout(title=f'{title_text} (Sin datos para filtros)')

    cols_existentes = [col for col in columnas_nomina_sumar if col in df_plot.columns]
    if not cols_existentes: return fig.update_layout(title=f'{title_text} (Columnas de nómina no encontradas)')
    
    df_plot['Nomina_Filtrada'] = df_plot[cols_existentes].sum(axis=1)
    if df_plot['Nomina_Filtrada'].sum() == 0 : return fig.update_layout(title=f'{title_text} (Suma de nómina es cero)')

    x_axis_plot, x_axis_label, color_axis_label = 'MARCA', 'Marca', 'MARCA'
    groupby_cols = ['MARCA']
    title_text = 'Nómina General'

    if marca_sel_std:
        title_text = f'Nómina: {marca_sel_std}'
        groupby_cols = ['UBICACION']
        x_axis_plot, x_axis_label, color_axis_label = 'UBICACION', 'Ubicación', 'UBICACION'
        if ubicacion_sel_std:
            title_text += f' - {ubicacion_sel_std}'
            groupby_cols = ['MES']
            x_axis_plot, x_axis_label, color_axis_label = 'MES', 'Mes', 'UBICACION' 
            if mes_sel_std: title_text += f' ({mes_sel_std})'
    
    valid_groupby_cols = [col for col in groupby_cols if col in df_plot.columns]
    if not valid_groupby_cols : 
        df_agrupado = pd.DataFrame({'Nomina_Filtrada': [df_plot['Nomina_Filtrada'].sum()]})
        df_agrupado['Concepto'] = "Total General"
        x_axis_plot, x_axis_label = 'Concepto', 'Concepto'
        color_axis_label = None
    else:
         df_agrupado = df_plot.groupby(valid_groupby_cols, as_index=False)['Nomina_Filtrada'].sum()
         if x_axis_plot == 'MES' and 'MES' in df_agrupado.columns: df_agrupado = df_agrupado.sort_values(by='MES')
    
    if df_agrupado.empty or df_agrupado['Nomina_Filtrada'].sum() == 0: 
        return fig.update_layout(title=f'{title_text} (No hay datos para agrupar o suma es cero)')
    
    fig = px.bar(df_agrupado, x=x_axis_plot, y='Nomina_Filtrada', 
                 color=color_axis_label if color_axis_label and color_axis_label in df_agrupado.columns else None, 
                 barmode='group', title=title_text, labels={'Nomina_Filtrada': 'Nómina ($)'}, text='Nomina_Filtrada')
    fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
    fig.update_layout(plot_bgcolor='white', paper_bgcolor='white', font_color='#333', legend_title_text='Leyenda', hovermode="x unified", xaxis_title=x_axis_label)
    return fig

@app.callback(Output('ventas-bar-chart', 'figure'), [Input('marca-dropdown', 'value'), Input('ubicacion-dropdown', 'value'), Input('mes-dropdown', 'value')])
def actualizar_grafico_ventas(marca_seleccionada, ubicacion_seleccionada, mes_seleccionado):
    df_plot = df_ventas.copy() if df_ventas is not None and not df_ventas.empty else pd.DataFrame()
    title_text = 'Ventas'
    fig = go.Figure(layout={'title': f'{title_text} (No hay datos)', 'xaxis': {'visible': False}, 'yaxis': {'visible': False}})
    if df_plot.empty: return fig

    marca_sel_std = str(marca_seleccionada).upper().strip() if marca_seleccionada else None
    ubicacion_sel_std = str(ubicacion_seleccionada).upper().strip() if ubicacion_seleccionada else None
    mes_sel_std = str(mes_seleccionado).strip() if mes_seleccionado else None

    if marca_sel_std and 'Marca' in df_plot.columns: df_plot = df_plot[df_plot['Marca'] == marca_sel_std]
    if ubicacion_sel_std and 'UBICACION' in df_plot.columns: df_plot = df_plot[df_plot['UBICACION'] == ubicacion_sel_std]
    if mes_sel_std and 'MES' in df_plot.columns: df_plot = df_plot[df_plot['MES'] == mes_sel_std]

    if 'Marca' in df_plot.columns: df_plot = df_plot[df_plot['Marca'] != 'DESCONOCIDA']
    if 'UBICACION' in df_plot.columns: df_plot = df_plot[df_plot['UBICACION'] != 'DESCONOCIDA']

    if df_plot.empty: return fig.update_layout(title=f'{title_text} (Sin datos para filtros)')
    if 'Ventas $' not in df_plot.columns or df_plot['Ventas $'].sum() == 0: 
        return fig.update_layout(title=f'{title_text} (Columna Ventas $ no encontrada o suma es cero)')

    x_axis_plot, x_axis_label, color_axis_label = 'Marca', 'Marca', 'Marca'
    groupby_cols = ['Marca']
    title_text = 'Ventas Generales'

    if marca_sel_std:
        title_text = f'Ventas: {marca_sel_std}'
        groupby_cols = ['UBICACION']
        x_axis_plot, x_axis_label, color_axis_label = 'UBICACION', 'Ubicación', 'UBICACION'
        if ubicacion_sel_std:
            title_text += f' - {ubicacion_sel_std}'
            groupby_cols = ['MES']
            x_axis_plot, x_axis_label, color_axis_label = 'MES', 'Mes', 'UBICACION'
            if mes_sel_std: title_text += f' ({mes_sel_std})'

    valid_groupby_cols = [col for col in groupby_cols if col in df_plot.columns]
    if not valid_groupby_cols:
        df_agrupado = pd.DataFrame({'Ventas_Filtradas': [df_plot['Ventas $'].sum()]})
        df_agrupado['Concepto'] = "Total General"
        x_axis_plot, x_axis_label = 'Concepto', 'Concepto'
        color_axis_label = None
    else:
        df_agrupado = df_plot.groupby(valid_groupby_cols, as_index=False)['Ventas $'].sum().rename(columns={'Ventas $': 'Ventas_Filtradas'})
        if x_axis_plot == 'MES' and 'MES' in df_agrupado.columns: df_agrupado = df_agrupado.sort_values(by='MES')
        
    if df_agrupado.empty or df_agrupado['Ventas_Filtradas'].sum() == 0: 
        return fig.update_layout(title=f'{title_text} (No hay datos para agrupar o suma es cero)')

    fig = px.bar(df_agrupado, x=x_axis_plot, y='Ventas_Filtradas', 
                 color=color_axis_label if color_axis_label and color_axis_label in df_agrupado.columns else None, 
                 barmode='group', title=title_text, labels={'Ventas_Filtradas': 'Ventas ($)'}, text='Ventas_Filtradas')
    fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
    fig.update_layout(plot_bgcolor='white', paper_bgcolor='white', font_color='#333', legend_title_text='Leyenda', hovermode="x unified", xaxis_title=x_axis_label)
    return fig

@app.callback(Output('scatter-chart', 'figure'), [Input('marca-dropdown', 'value'), Input('ubicacion-dropdown', 'value'), Input('mes-dropdown', 'value')])
def actualizar_grafico_dispersion(marca_seleccionada, ubicacion_seleccionada, mes_seleccionado):
    df_v_filt = df_ventas.copy() if df_ventas is not None and not df_ventas.empty else pd.DataFrame()
    df_n_filt = df_nomina.copy() if df_nomina is not None and not df_nomina.empty else pd.DataFrame()
    
    marca_sel_std = str(marca_seleccionada).upper().strip() if marca_seleccionada else None
    ubicacion_sel_std = str(ubicacion_seleccionada).upper().strip() if ubicacion_seleccionada else None
    mes_sel_std = str(mes_seleccionado).strip() if mes_seleccionado else None

    title_suffix = f" ({marca_sel_std if marca_sel_std else 'General'}"
    fig = go.Figure(layout={'title': f'Relación Nómina/Margen vs Ventas{title_suffix}) (No hay datos)'})
    if df_v_filt.empty or df_n_filt.empty: return fig

    if mes_sel_std:
        if 'MES' in df_v_filt.columns: df_v_filt = df_v_filt[df_v_filt['MES'] == mes_sel_std]
        if 'MES' in df_n_filt.columns: df_n_filt = df_n_filt[df_n_filt['MES'] == mes_sel_std]
        title_suffix += f", Mes: {mes_sel_std}"
    else: title_suffix += ", Todos los Meses"
    title_suffix += ")"
    
    if 'Marca' in df_v_filt.columns: df_v_filt = df_v_filt[df_v_filt['Marca'] != 'DESCONOCIDA']
    if 'UBICACION' in df_v_filt.columns: df_v_filt = df_v_filt[df_v_filt['UBICACION'] != 'DESCONOCIDA']
    if 'MARCA' in df_n_filt.columns: df_n_filt = df_n_filt[df_n_filt['MARCA'] != 'DESCONOCIDA']
    if 'UBICACION' in df_n_filt.columns: df_n_filt = df_n_filt[df_n_filt['UBICACION'] != 'DESCONOCIDA']

    df_v_scatter = pd.DataFrame()
    if not df_v_filt.empty and all(c in df_v_filt for c in ['Marca', 'UBICACION', 'Ventas $', 'Coste $']):
      df_v_scatter = df_v_filt.groupby(['Marca', 'UBICACION'], as_index=False).agg(Ventas_Marca_Ubicacion=('Ventas $', 'sum'), Coste_Ventas_Marca_Ubicacion=('Coste $', 'sum'))
      if not df_v_scatter.empty: df_v_scatter['Margen_Bruto_Marca_Ubicacion'] = df_v_scatter['Ventas_Marca_Ubicacion'] - df_v_scatter['Coste_Ventas_Marca_Ubicacion']

    df_n_scatter = pd.DataFrame()
    if not df_n_filt.empty and all(c in df_n_filt for c in ['MARCA', 'UBICACION']):
      cols_n_scatter = [col for col in columnas_nomina_sumar if col in df_n_filt.columns]
      if cols_n_scatter:
          df_n_scatter_temp = df_n_filt.groupby(['MARCA', 'UBICACION'], as_index=False)[cols_n_scatter].sum()
          if not df_n_scatter_temp.empty:
              df_n_scatter_temp['Nomina_Marca_Ubicacion'] = df_n_scatter_temp[cols_n_scatter].sum(axis=1)
              df_n_scatter = df_n_scatter_temp.rename(columns={'MARCA': 'Marca'})[['Marca', 'UBICACION', 'Nomina_Marca_Ubicacion']]

    if marca_sel_std:
        if 'Marca' in df_v_scatter.columns: df_v_scatter = df_v_scatter[df_v_scatter['Marca'] == marca_sel_std]
        if 'Marca' in df_n_scatter.columns: df_n_scatter = df_n_scatter[df_n_scatter['Marca'] == marca_sel_std]
    if ubicacion_sel_std:
        if 'UBICACION' in df_v_scatter.columns: df_v_scatter = df_v_scatter[df_v_scatter['UBICACION'] == ubicacion_sel_std]
        if 'UBICACION' in df_n_scatter.columns: df_n_scatter = df_n_scatter[df_n_scatter['UBICACION'] == ubicacion_sel_std]
    
    df_combinado = pd.DataFrame()
    if not df_v_scatter.empty and not df_n_scatter.empty and 'Marca' in df_v_scatter and 'UBICACION' in df_v_scatter and 'Marca' in df_n_scatter and 'UBICACION' in df_n_scatter:
        df_combinado = pd.merge(df_n_scatter, df_v_scatter, on=['Marca', 'UBICACION'], how='inner')
    
    if df_combinado.empty: return fig.update_layout(title=f'Relación Nómina/Margen vs Ventas{title_suffix} (No hay datos combinados)', annotations=[{'text': 'No hay datos para los filtros.', 'showarrow': False, 'font':{'size':10}}])

    fig = make_subplots(rows=1, cols=2, subplot_titles=('Nómina vs. Ventas', 'Margen vs. Ventas'), column_widths=[0.5, 0.5])
    text_labels = df_combinado.apply(lambda row: f"{row.get('Marca', '')}<br>{row.get('UBICACION', '')}", axis=1)
    customdata = df_combinado[['Marca', 'UBICACION']].values if all(c in df_combinado for c in ['Marca', 'UBICACION']) else None

    if 'Nomina_Marca_Ubicacion' in df_combinado and 'Ventas_Marca_Ubicacion' in df_combinado:
        fig.add_trace(go.Scatter(x=df_combinado['Nomina_Marca_Ubicacion'], y=df_combinado['Ventas_Marca_Ubicacion'], mode='markers+text', text=text_labels, textposition="top center", customdata=customdata, hovertemplate='<b>%{customdata[0]}</b><br>%{customdata[1]}<br>Nómina: %{x:,.0f}<br>Ventas: %{y:,.0f}<extra></extra>'), row=1, col=1)
    if 'Margen_Bruto_Marca_Ubicacion' in df_combinado and 'Ventas_Marca_Ubicacion' in df_combinado:
        fig.add_trace(go.Scatter(x=df_combinado['Margen_Bruto_Marca_Ubicacion'], y=df_combinado['Ventas_Marca_Ubicacion'], mode='markers+text', text=text_labels, textposition="top center", customdata=customdata, hovertemplate='<b>%{customdata[0]}</b><br>%{customdata[1]}<br>Margen: %{x:,.0f}<br>Ventas: %{y:,.0f}<extra></extra>'), row=1, col=2)
    
    fig.update_layout(title_text=f'Nómina/Margen vs Ventas por Ubicación{title_suffix}', showlegend=False, plot_bgcolor='white', paper_bgcolor='white', font_color='#333')
    fig.update_xaxes(title_text='Nómina ($)', row=1, col=1); fig.update_yaxes(title_text='Ventas ($)', row=1, col=1)
    fig.update_xaxes(title_text='Margen Bruto ($)', row=1, col=2); fig.update_yaxes(title_text='Ventas ($)', row=1, col=2)
    return fig

# 6. Ejecución de la Aplicación
if __name__ == '__main__':
    # Cambiar el puerto si el 8055 o 8057 están ocupados
    app.run(debug=True, port=8061) 



Cargando todas las hojas de: CONTROL ALQUILER.xlsx
  - Procesando hoja: W´S
    Columnas encontradas en hoja 'W´S': ['MARCA', 'MES', 'UBICACION', 'Concepto', 'N° de Factura', 'PERIODO', 'MONTO BASE', 'VENTA', 'Status', 'Fecha de Pago']
  - Procesando hoja: ZIPPY5
    Columnas encontradas en hoja 'ZIPPY5': ['MARCA', 'RAZON SOCIAL', 'MES', 'UBICACION', 'Concepto', 'N° de Factura', 'PERIODO', 'MONTO BASE', 'VENTA', 'Status', 'Fecha de Pago']
  - Procesando hoja: TMB
    Columnas encontradas en hoja 'TMB': ['MARCA', 'RAZON SOCIAL', 'MES', 'UBICACION', 'Concepto', 'N° de Factura', 'PERIODO', 'MONTO BASE', 'VENTA', 'Status', 'Fecha de Pago']
  - Procesando hoja: VOGA4
    Columnas encontradas en hoja 'VOGA4': ['MARCA', 'RAZON SOCIAL', 'MES', 'UBICACION', 'Concepto', 'N° de Factura', 'PERIODO', 'MONTO BASE', 'VENTA', 'Status', 'Fecha de Pago']
  - Procesando hoja: SWK
    Columnas encontradas en hoja 'SWK': ['MARCA', 'RAZON SOCIAL', 'MES', 'UBICACION', 'Concepto', 'N° de Factura', 'PERIODO', 

C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning:

Data Validation extension is not supported and will be removed

C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning:

Data Validation extension is not supported and will be removed



Archivo 'BBDD_VENTAS.xlsx' cargado exitosamente (primera hoja).
Archivo 'ALCALDIAS.xlsx' cargado exitosamente (primera hoja).


C:\ProgramData\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:85: UserWarning:

Data Validation extension is not supported and will be removed



Archivo '1-Gastos de Personal x  Razón Social 2025.xlsx' cargado exitosamente (primera hoja).

--- DataFrames cargados en el diccionario 'dataframes' ---
dict_keys(['CONTROL ALQUILER.xlsx', 'CONDOMINIOS.xlsx', 'BBDD_VENTAS.xlsx', 'ALCALDIAS.xlsx', '1-Gastos de Personal x\xa0 Razón Social 2025.xlsx'])

--- Procesando DataFrame 'BBDD_VENTAS.xlsx' ---

--- Procesando DataFrame COMBINADO 'CONTROL ALQUILER.xlsx' ---
DEBUG ALQUILER: Marca después de fillna/upper/strip: ['DESCONOCIDA' 'WOMEN SECRET' 'ZIPPY' 'TIMBERLAND' 'VOGA' 'SWAROVSKI'
 'PUSH&CO' 'SPRINGFIELD' 'PLANETA SPORT' 'PARFOIS']
DEBUG ALQUILER (Combinado): df_alquiler_agrupado['Monto_Alquiler_Total'].sum(): 1658139.9691
DEBUG ALQUILER (Combinado): Marcas únicas en agrupado: ['ALDO' 'BALU' 'BIMBA Y LOLA' 'CAROLINA HERRERA' 'CORTEFIEL' 'DESCONOCIDA'
 'MINISO' 'MNG' 'PARFOIS' 'PLANETA SPORT' 'PUSH&CO' 'SCALPERS'
 'SPRINGFIELD' 'SWAROVSKI' 'TIMBERLAND' 'VOGA' 'WOMEN SECRET' 'ZIPPY']


C:\Users\avillarreal\AppData\Local\Temp\ipykernel_15272\242890661.py:310: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\avillarreal\AppData\Local\Temp\ipykernel_15272\242890661.py:312: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(v

In [ ]:
df_alquiler_agrupado